# pRF Stimulus Video Generator

Creates 60 Hz videos from .mat files containing visual stimuli (Bars, Rings, Wedges).

**Specifications:**
- Input resolution: 102 pixels (width) × 77 pixels (height)
- Output resolution: 102 × 102 pixels (black margins added top/bottom)
- Frame rate: 60 Hz
- Duration per image: 2 seconds (120 frames per image)
- Input: s.im from .mat files (102 × 77 × 120)
- Output: Individual MP4 videos + Combined video (Bars × 2, Wedges, Rings)

In [1]:
# General imports
import os
import cv2
import numpy as np
import scipy.io as sio

In [2]:
# Define parameters
fps = 60  # Frame rate in Hz
duration_per_frame = 2  # Duration each image is displayed (seconds)
frames_per_image = fps * duration_per_frame  # 120 frames per image

# Expected dimensions from s.im
expected_width = 102
expected_height = 77
expected_n_images = 120

# Output dimensions (square)
output_size = 102

# Input/output paths
input_dir = '/home/mszinte/disks/meso_S/data/amblyo7T_prf/derivatives/vdm/original/'  # Directory containing .mat files
output_dir = '/home/mszinte/disks/meso_S/data/amblyo7T_prf/derivatives/vdm/original/'  # Output directory for videos

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Files to process
mat_files = ['Bars.mat', 'Rings.mat', 'Wedges.mat']

In [3]:
# Process each .mat file
for filename in mat_files:
    print(f"\nProcessing {filename}...")
    
    # Load .mat file
    mat_path = os.path.join(input_dir, filename)
    mat = sio.loadmat(mat_path)
    s_im = mat['s']['im'][0, 0]  # Extract s.im
    
    # Get dimensions
    width, height, n_images = s_im.shape
    print(f"  Loaded s.im with shape: {s_im.shape}")
    print(f"  Expected shape: ({expected_width}, {expected_height}, {expected_n_images})")
    
    # Verify dimensions
    assert width == expected_width, f"Width mismatch: {width} != {expected_width}"
    assert height == expected_height, f"Height mismatch: {height} != {expected_height}"
    assert n_images == expected_n_images, f"N images mismatch: {n_images} != {expected_n_images}"
    
    # Calculate padding to make square (102×102)
    height_to_add = (output_size - height) // 2  # (102 - 77) // 2 = 12 pixels each side
    remaining_pixels = (output_size - height) % 2  # Handle odd difference
    
    print(f"  Adding {height_to_add} pixels margin top and bottom (+ {remaining_pixels} extra at bottom)")
    print(f"  Output size: {output_size}×{output_size}")
    
    # Create video writer with square dimensions
    video_name = filename.replace('.mat', '_60hz.mp4')
    output_path = os.path.join(output_dir, video_name)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (output_size, output_size), False)
    
    # Write each image 120 times (2 seconds at 60 Hz)
    for i in range(n_images):
        # Get frame (width × height) = (102 × 77)
        frame = s_im[:, :, i]
        
        # Convert from 0-1 float to 0-255 uint8
        frame_uint8 = (frame * 255).astype(np.uint8)
        
        # Transpose to match OpenCV format (height × width) = (77 × 102)
        frame_cv = frame_uint8.T
        
        # Add black margins top and bottom to make 102×102
        top_margin = np.zeros((height_to_add, width), dtype=np.uint8)
        bottom_margin = np.zeros((height_to_add + remaining_pixels, width), dtype=np.uint8)
        frame_padded = np.vstack([top_margin, frame_cv, bottom_margin])  # Now 102 × 102
        
        # Write this frame 120 times
        for _ in range(frames_per_image):
            out.write(frame_padded)
        
        # Progress indicator
        if (i + 1) % 20 == 0:
            print(f"  Progress: {i + 1}/{n_images} images written")
    
    out.release()
    
    total_duration = n_images * duration_per_frame
    total_frames = n_images * frames_per_image
    print(f"  ✓ Created {video_name}")
    print(f"    Resolution: {output_size}×{output_size} pixels")
    print(f"    Duration: {total_duration} seconds")
    print(f"    Total frames: {total_frames}")

print("\n=== All individual videos created successfully! ===")


Processing Bars.mat...
  Loaded s.im with shape: (102, 77, 120)
  Expected shape: (102, 77, 120)
  Adding 12 pixels margin top and bottom (+ 1 extra at bottom)
  Output size: 102×102
  Progress: 20/120 images written
  Progress: 40/120 images written
  Progress: 60/120 images written
  Progress: 80/120 images written
  Progress: 100/120 images written
  Progress: 120/120 images written
  ✓ Created Bars_60hz.mp4
    Resolution: 102×102 pixels
    Duration: 240 seconds
    Total frames: 14400

Processing Rings.mat...
  Loaded s.im with shape: (102, 77, 120)
  Expected shape: (102, 77, 120)
  Adding 12 pixels margin top and bottom (+ 1 extra at bottom)
  Output size: 102×102
  Progress: 20/120 images written
  Progress: 40/120 images written
  Progress: 60/120 images written
  Progress: 80/120 images written
  Progress: 100/120 images written
  Progress: 120/120 images written
  ✓ Created Rings_60hz.mp4
    Resolution: 102×102 pixels
    Duration: 240 seconds
    Total frames: 14400

Pro

In [4]:
# Create combined video: Bars × 2, Wedges × 1, Rings × 1
print("\n=== Creating combined video ===")
print("Sequence: Bars → Bars → Rings → Wedges")

# Load all three .mat files
bars_mat = sio.loadmat(os.path.join(input_dir, 'Bars.mat'))
rings_mat = sio.loadmat(os.path.join(input_dir, 'Rings.mat'))
wedges_mat = sio.loadmat(os.path.join(input_dir, 'Wedges.mat'))

bars_im = bars_mat['s']['im'][0, 0]
rings_im = rings_mat['s']['im'][0, 0]
wedges_im = wedges_mat['s']['im'][0, 0]

# Create combined video writer
combined_video_name = 'Bars_Bars_Rings_Wedges_60hz.mp4'
combined_output_path = os.path.join(output_dir, combined_video_name)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_combined = cv2.VideoWriter(combined_output_path, fourcc, fps, (output_size, output_size), False)

# Calculate padding
height_to_add = (output_size - expected_height) // 2
remaining_pixels = (output_size - expected_height) % 2

# Sequence: Bars, Bars, Wedges, Rings
sequence = [bars_im, bars_im, rings_im, wedges_im]
sequence_names = ['Bars (1st)', 'Bars (2nd)', 'Rings', 'Wedges']

total_frames_written = 0

for seq_idx, (s_im, seq_name) in enumerate(zip(sequence, sequence_names)):
    print(f"\nAdding {seq_name}...")
    n_images = s_im.shape[2]
    
    for i in range(n_images):
        # Get frame
        frame = s_im[:, :, i]
        
        # Convert and transpose
        frame_uint8 = (frame * 255).astype(np.uint8)
        frame_cv = frame_uint8.T
        
        # Add margins
        top_margin = np.zeros((height_to_add, expected_width), dtype=np.uint8)
        bottom_margin = np.zeros((height_to_add + remaining_pixels, expected_width), dtype=np.uint8)
        frame_padded = np.vstack([top_margin, frame_cv, bottom_margin])
        
        # Write this frame 120 times
        for _ in range(frames_per_image):
            out_combined.write(frame_padded)
            total_frames_written += 1
        
        # Progress indicator
        if (i + 1) % 20 == 0:
            print(f"  Progress: {i + 1}/{n_images} images written")

out_combined.release()

total_duration_combined = total_frames_written / fps
print(f"\n✓ Created {combined_video_name}")
print(f"  Resolution: {output_size}×{output_size} pixels")
print(f"  Total frames: {total_frames_written}")
print(f"  Duration: {total_duration_combined:.1f} seconds ({total_duration_combined/60:.1f} minutes)")
print("\n=== All videos created successfully! ===")


=== Creating combined video ===
Sequence: Bars → Bars → Rings → Wedges

Adding Bars (1st)...
  Progress: 20/120 images written
  Progress: 40/120 images written
  Progress: 60/120 images written
  Progress: 80/120 images written
  Progress: 100/120 images written
  Progress: 120/120 images written

Adding Bars (2nd)...
  Progress: 20/120 images written
  Progress: 40/120 images written
  Progress: 60/120 images written
  Progress: 80/120 images written
  Progress: 100/120 images written
  Progress: 120/120 images written

Adding Rings...
  Progress: 20/120 images written
  Progress: 40/120 images written
  Progress: 60/120 images written
  Progress: 80/120 images written
  Progress: 100/120 images written
  Progress: 120/120 images written

Adding Wedges...
  Progress: 20/120 images written
  Progress: 40/120 images written
  Progress: 60/120 images written
  Progress: 80/120 images written
  Progress: 100/120 images written
  Progress: 120/120 images written

✓ Created Bars_Bars_Ring